In [ ]:
# User-defined variables
work_dir = '/Users/Tem/Documents/naturalistic-threat-ptsd'
npy_dir = f"{work_dir}/data/neural-naturalistic/niftis/masked/subcort-harvard-oxford-6mm" #directory housing masked npy arrays
mask_dir = f"{work_dir}/masks/subcort-harvard-oxford"
nPerms = 10000
trim_TRs = 4 #number of TRs to trim from beginning and end of functional scan (separate from hemodynamic adjustment)

In [ ]:
# Import packages
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import levene, mannwhitneyu
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
from scipy.stats import iqr
from statsmodels.formula.api import ols
import statsmodels.api as sm
from datetime import datetime
import random
from sklearn.metrics import pairwise_distances
from scipy.stats import rankdata
from nltools.data import Adjacency
from statsmodels.stats.multitest import fdrcorrection
from matplotlib import gridspec

In [ ]:
# Load movie metadata 
movie_metadata = pd.read_csv(f"{work_dir}/data/task/stim-metadata-naturalistic.csv") 
movie_metadata['Start_TR'][0] = movie_metadata['Start_TR'][0] + trim_TRs
movie_metadata['Stop_TR'][len(movie_metadata) - 1] = movie_metadata['Stop_TR'][len(movie_metadata) - 1] - trim_TRs
movie_metadata['Scene_Length'] = movie_metadata['Stop_TR'] - movie_metadata['Start_TR']

nScenes = len(movie_metadata)
movie_TRs = [list(range(movie_metadata['Start_TR'][i], movie_metadata['Stop_TR'][i])) for i in range(0, nScenes)]

In [ ]:
# Load and define participant data
symptom_labels = ['Re-experiencing', 'Avoidance-Numbing', 'Hyperarousal']
CAPS_data = pd.read_csv(f"{work_dir}/data/subs/CAPS.csv")
cohort_IDs = pd.read_csv(f"{work_dir}/data/subs/cohort_IDs.csv")

# Load ROI labels
ROI_labels = pd.read_csv(f"{mask_dir}/ROI-labels.csv") #directory housing masks & labels

# Define n's
n_regions = len(ROI_labels)
nCohort = len(cohort_IDs)

In [ ]:
# Loop through ROIs to calculate interSC, classification, and correlation with symptoms using child script
counter = 0
for ROI_no in range(n_regions):
    disp_name = ROI_labels['label'][ROI_no]
    ROI_ID = ROI_labels['index'][ROI_no]
    cROI = f"HO_{ROI_ID}"
    print(f"Running Harvard-Oxford ROI #{ROI_ID}, {ROI_labels['label'][counter]}...")
    startTime = datetime.now()
    
    %run './07. run-naturalistic-child.ipynb'
    ROI_values = pd.Series(ROI_values, index = ROI_value_labels)              
    
    if counter == 0:
        all_ROI_values = pd.DataFrame(columns = ROI_value_labels)
        all_ROI_values = all_ROI_values.append(ROI_values, ignore_index = True)
    else: 
        all_ROI_values = all_ROI_values.append(ROI_values, ignore_index = True)
        
    print(f"Runtime for ROI #{ROI_ID}: {datetime.now() - startTime}\n")    
    counter = counter + 1

In [ ]:
all_ROI_values.insert(0, 'ROI', ROI_labels['label'])
date_string = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
all_ROI_values.to_csv(f"{work_dir}/data/tables/naturalistic-{nPerms}_perms-{date_string}.csv", index = False)
all_ROI_values